# Model complexity, context specification and assembly policies
In this demo we explore the effects of specified conditions on Agents (e.g. bound conditions, modification conditions)  and assembly policies on the combinatorial complexity of dynamical models.

First, we import INDRA's TRIPS input API and PySB model assembler.

In [1]:
from indra import trips
from indra.assemblers import PysbAssembler

## Model1: RAS to ERK without specifying agent context
In the first case, two binding events and a phosphorylation is described with no additional context specified on any of the proteins.

In [16]:
tp = trips.process_text('RAS binds RAF and RAF binds MEK. MEK phosphorylates ERK.')

This yields 3 INDRA Statements, as follows. Here empty parentheses after the Agent names indicate that there is no additional context specified on them.

In [3]:
tp.statements

[Complex(RAS(), RAF()), Complex(RAF(), MEK()), Phosphorylation(MEK(), ERK())]

### Assembly with one-step policy
We now assemble this model using the default `one_step` policy and store it in the `model1_one` variable.

In [4]:
pa = PysbAssembler()
pa.add_statements(tp.statements)
pa.make_model(policies='one_step')

<Model 'None' (monomers: 4, rules: 5, parameters: 9, expressions: 0, compartments: 0) at 0x7f7fe3929290>

In [5]:
model1_one = pa.model

The model has 4 Monomers and 5 Rules.

In [6]:
model1_one.monomers

ComponentSet([
 Monomer(u'RAF', [u'ras', u'map2k']),
 Monomer(u'RAS', [u'map3k']),
 Monomer(u'MEK', [u'map3k']),
 Monomer(u'ERK', [u'phospho'], {u'phospho': [u'u', u'p']}),
 ])

In [7]:
model1_one.rules

ComponentSet([
 Rule(u'RAS_RAF_bind', RAS(map3k=None) + RAF(ras=None) >> RAS(map3k=1) % RAF(ras=1), kf_rr_bind_1),
 Rule(u'RAS_RAF_dissociate', RAS(map3k=1) % RAF(ras=1) >> RAS(map3k=None) + RAF(ras=None), kr_rr_bind_1),
 Rule(u'RAF_MEK_bind', RAF(map2k=None) + MEK(map3k=None) >> RAF(map2k=1) % MEK(map3k=1), kf_rm_bind_1),
 Rule(u'RAF_MEK_dissociate', RAF(map2k=1) % MEK(map3k=1) >> RAF(map2k=None) + MEK(map3k=None), kr_rm_bind_1),
 Rule(u'MEK_phosphorylation_ERK_phospho', MEK() + ERK(phospho=u'u') >> MEK() + ERK(phospho=u'p'), kf_me_phosphorylation_1),
 ])

Let's examine the last rule which corresponds to MEK phosphorylating ERK. Here, `MEK()` appears without additional context specified. This means that the rule will apply to **any** form of `MEK`, for instance, MEK that is bound to RAF. 

We now generate the rule-based model into a reaction network form using PySB's interface to BioNetGen.

In [8]:
from pysb.bng import generate_equations
generate_equations(model1_one)

Let's now inspect each individual species that was created by the reaction network generation.

In [9]:
model1_one.species

[RAF(ras=None, map2k=None),
 RAS(map3k=None),
 MEK(map3k=None),
 ERK(phospho='u'),
 RAF(ras=1, map2k=None) % RAS(map3k=1),
 MEK(map3k=1) % RAF(ras=None, map2k=1),
 ERK(phospho='p'),
 MEK(map3k=1) % RAF(ras=2, map2k=1) % RAS(map3k=2)]

We see that MEK appears in 3 distinct forms:
- MEK(map3k=None): MEK not bound to anything
- MEK(map3k=1) % RAF(ras=None, map2k=1): MEK bound to RAF which is not bound to RAS
- MEK(map3k=1) % RAF(ras=2, map2k=1) % RAS(map3k=2): MEK bounds to RAF that is bound to RAS

As explained above, the rule `MEK_phosphorylation_ERK_phospho` will apply to all 3 of these forms of MEK.

### Assembly with two-step policy
Let's now assemble the same model with the `two-step` policy. This will result in a model detailed model in which MEK first binds ERK reversibly, and phosphorylated ERK is released from the MEK-ERK complex. We will store this model in the `model1_two` variable.

In [10]:
pa.make_model(policies='two_step')

<Model 'None' (monomers: 4, rules: 7, parameters: 11, expressions: 0, compartments: 0) at 0x7f7fe39292d0>

In [11]:
model1_two = pa.model

In [12]:
model1_two.monomers

ComponentSet([
 Monomer(u'RAF', [u'ras', u'map2k']),
 Monomer(u'RAS', [u'map3k']),
 Monomer(u'MEK', [u'map3k', u'mapk']),
 Monomer(u'ERK', [u'phospho', u'map2k'], {u'phospho': [u'u', u'p']}),
 ])

In [13]:
model1_two.rules

ComponentSet([
 Rule(u'RAS_RAF_bind', RAS(map3k=None) + RAF(ras=None) >> RAS(map3k=1) % RAF(ras=1), kf_rr_bind_1),
 Rule(u'RAS_RAF_dissociate', RAS(map3k=1) % RAF(ras=1) >> RAS(map3k=None) + RAF(ras=None), kr_rr_bind_1),
 Rule(u'RAF_MEK_bind', RAF(map2k=None) + MEK(map3k=None) >> RAF(map2k=1) % MEK(map3k=1), kf_rm_bind_1),
 Rule(u'RAF_MEK_dissociate', RAF(map2k=1) % MEK(map3k=1) >> RAF(map2k=None) + MEK(map3k=None), kr_rm_bind_1),
 Rule(u'MEK_phosphorylation_bind_ERK_phospho', MEK(mapk=None) + ERK(phospho=u'u', map2k=None) >> MEK(mapk=1) % ERK(phospho=u'u', map2k=1), kf_me_bind_1),
 Rule(u'MEK_phosphorylation_ERK_phospho', MEK(mapk=1) % ERK(phospho=u'u', map2k=1) >> MEK(mapk=None) + ERK(phospho=u'p', map2k=None), kc_me_phosphorylation_1),
 Rule(u'MEK_dissoc_ERK', MEK(mapk=1) % ERK(map2k=1) >> MEK(mapk=None) + ERK(map2k=None), kr_me_bind_1),
 ])

We can now generate the reaction network for `model2` and inspect the individual species that are created.

In [14]:
generate_equations(model1_two)

In [19]:
model1_two.species

NameError: name 'model1_two' is not defined

The two-step policy produced a total of 11 molecular species and ERK now appears in 5 possible forms:
- ERK(phospho='u', map2k=None): unphosphorylated ERK not bound to MEK
- ERK(phospho='u', map2k=1) % MEK(map3k=None, mapk=1): unphosphorylated ERK bound to MEK
- ERK(phospho='u', map2k=1) % MEK(map3k=2, mapk=1) % RAF(ras=None, map2k=2): unphosphorylated ERK bound to MEK which is bound to RAF
-  ERK(phospho='u', map2k=1) % MEK(map3k=2, mapk=1) % RAF(ras=3, map2k=2) % RAS(map3k=3): unphosphorylated ERK bound to MEK which is bound to RAF which, in turn, is bound to RAS
- ERK(phospho='p', map2k=None): phosphorylated ERK not bound to MEK